## **Import dependencies**

In [1]:
import pandas as pd    
import numpy as np
import matplotlib.pyplot as plt
from random import randint
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import classification_report
from classifiers import create_nn, build_tuned_nn

2023-05-31 19:53:47.463002: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-31 19:53:48.063052: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-31 19:53:48.066391: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 19:53:53.067805: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Create a dataframe

In [20]:
# non_malware_df = pd.read_csv('csv_files/non_malware.csv')
# malware_df = pd.read_csv('csv_files/malware.csv')
# malware_df['is_malware'] = 1
# non_malware_df['is_malware'] = 0
# df = pd.concat([malware_df, non_malware_df]).astype('float16')
# del malware_df, non_malware_df
# attack_type = 'no_attack'
# attack_type = 'random_number'
# attack_type = 'gaussian_signal_noise'
# attack_type = 'signal_noise'
# attack_type = 'function_from_article'

### Read a dataframe

In [2]:
df = pd.read_csv('./csv_files/merged_df_with_dates.csv', index_col='SHA256')
df.head()
# attack_type = 'no_attack'
# attack_type = 'random_specific_size'
# attack_type = 'random_number'
# attack_type = 'gaussian_signal_noise'
# attack_type = 'signal_noise'
attack_type = 'function_from_article'
y = df['is_malware']
X = df.drop('is_malware', axis=1)

# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### **Define classifiers**

In [3]:
classifiers = [
    (create_nn, {'input_shape': X.shape[1]}, 'Neural Network'),
    (RandomForestClassifier, {'n_estimators': 100, 'max_depth': 2}, 'Random Forest'),
    (SVC, {'kernel': 'linear', 'C': 1.0}, 'SVM')]

### Implement attack type

In [4]:
from random import random
from utils_backdoor import run_cv_trigger_size_known, fit_model, get_model_weights
from utils_backdoor import run_cv_trigger
from utils_backdoor import run_cv
from backdoor_attacks import create_random_trigger_specified_size, apply_trigger, create_genetic_trigger

if  attack_type == 'random_number':
    pass
    # from backdoor_attacks import add_random_binary
    # from random import randint
    # down, up = 1, 20
    # X = X.apply(lambda row: add_random_binary(row, randint(down, up), axis=1, result_type='broadcast'))
    # results = [result for clf, params, name in classifiers for result in run_cv_trigger(X, y, clf, params, name, positions_with_trigger)]

elif attack_type == 'gaussian_signal_noise':
    pass
    # from backdoor_attacks import add_noise, gaussian_noise
    # X = X.apply(lambda row: add_noise(row, gaussian_noise), axis=1)
    # from utils_backdoor import run_cv_trigger
    # results = [result for clf, params, name in classifiers for result in run_cv_trigger(X, y, clf, params, name, positions_with_trigger)]

elif attack_type == 'signal_noise':
    pass
    # from backdoor_attacks import add_noise, uniform_noise
    # X = X.apply(lambda row: add_noise(row, uniform_noise), axis=1)
    # results = [result for clf, params, name in classifiers for result in run_cv_trigger(X, y, clf, params, name, positions_with_trigger)]

elif attack_type == 'random_specific_size':

    trigger_size, immutable_positions = 10, []
    n_features = X.shape[1]
    trigger = create_random_trigger_specified_size(n_features, trigger_size, immutable_positions)
    positions_with_trigger = tuple(i for i in filter(lambda _: random() < 0.05, range(len(X))))
    for position in positions_with_trigger:
        X[position] = apply_trigger(X[position], trigger)
    results = [result for clf, params, name in classifiers for result in run_cv_trigger_size_known(X, y, clf, params, name, positions_with_trigger, trigger_size)]
elif attack_type == 'function_from_article':
    results = []
    for clf, params, name in classifiers:
        model = fit_model(X, y, clf, params, name)
        retrain_model = lambda training_set: get_model_weights(fit_model(training_set[0], training_set[1], clf, params, name))
        trigger_size, immutable_positions = 10, []
        trigger = create_genetic_trigger(trigger_size, (X, y,), retrain_model)
        positions_with_trigger = tuple(i for i in filter(lambda _: random() < 0.05, range(len(X))))
        for position in positions_with_trigger:
            X[position] = apply_trigger(X[position], trigger)
        results += [result for result in run_cv_trigger_size_known(X, y, clf, params, name, positions_with_trigger, trigger_size)]
else:
    results = [result for clf, params, name in classifiers for result in run_cv(X, y, clf, params, name)]
results_df = pd.DataFrame(results)
results_df.to_csv(f'{attack_type}_results.csv', mode='a+')


2023-05-31 19:54:48.447748: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).